# Python Script for: Continuous foraging behavior shapes patch-leaving decisions in pigeons: A 3D tracking study

Guillermo Hidalgo Gadea

Optimal foraging behavior is a key component of successful adaptations to natural environments. Understanding how animals decide to stay near food or to leave it for another food patch gives us insights into the underlying cognitive mechanisms that govern adaptive behaviors. 3D pose tracking was used to determine how pigeons exploit a 4 square meter arena with two separate platforms (i.e. food patches) whose absolute and relative elevations were manipulated. Detailed kinematic features of foraging and traveling behaviors were quantified using automated video tracking, without a need for manual coding. Our computational approach captured continuous, high-dimensional movement patterns and enabled precise quantification of travel costs between patches. Combined with mixed-effects survival analysis, our detailed behavioral tracking provided unprecedented insight into the moment-by-moment dynamics of patch-leaving decisions of pigeons. As expected from behavior optimization models, our results showed a preference to visit a ground food platform first, and longer latencies to leave an elevated platform. Foraging activity significantly decreased throughout a session, with shorter visits, less pecks per visit, and a decrease in inter-peck variability. However, a mixed-effects Cox regression modeled pigeons' patch-leaving probability, demonstrating that current and cumulative foraging parameters between patches significantly enhanced the model's predictive power beyond patch accessibility (i.e., beyond travel costs). This suggests that pigeons integrate both current environmental cues and their individual foraging history when making patch-leaving decisions. Our findings are discussed in relation to the marginal value theorem and optimal foraging theory.

`updated 22.08.2025`

# Load Triangulation Environment

In [ ]:
# packages needed
import os, sys
import deeplabcut
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# load MotionPype
from motionpype import utils, dlcspecifics, aniposespecific

# Prepare Video Data

In [ ]:
# rename calibration and reference videos from path
calibration_path = r'G:\ForagingPlatformsArena_local\Calibration'
reference_path = r'G:\ForagingPlatformsArena_local\Reference'
projectname = 'ForagingPlatforms'
extension = '.avi'
rule1 = r'_|\\|/|.avi'
rule2 = '202' #common in all dates from 2022 to 2023
rule3 = 'P'
cam_assignement = { '20323040': 'camA',
                    '20323042': 'camB', 
                    '20323043': 'camC',
                    '20323044': 'camD', 
                    '20323049': 'camE', 
                    '20323052': 'camF'}

In [ ]:
keyword = 'calibration'
raw_videos = utils.scrapdirbystring(calibration_path, extension, output=False)

utils.batchrenamebyrules(raw_videos, calibration_path, extension, projectname, keyword, cam_assignement, rule1, rule2, rule3, rename = False)

In [ ]:
keyword = 'reference'
raw_videos = utils.scrapdirbystring(reference_path, extension, output=False)

utils.batchrenamebyrules(raw_videos, reference_path, extension, projectname, keyword, cam_assignement, rule1, rule2, rule3, rename = False)

# Create new Anipose Project fro 3D Triangulation

In [2]:
# create new anipose project
project_name = 'Triangulation'
project_directory = r'G:\ForagingPlatformsArena_local'
# see nested structure in session_list with Pigeon IDs as trials nested in dated sessions
session_list = [
                '20221019/P393', '20221019/P430', '20221019/P434', '20221020/P195', '20221020/P437', '20221020/P600',
                '20221021/P195', '20221021/P393', '20221021/P430', '20221021/P434', '20221021/P437', '20221021/P600',
                '20221024/P195', '20221024/P393', '20221024/P430', '20221024/P434', '20221024/P437', '20221024/P600',
                '20221025/P195', '20221025/P393', '20221025/P430', '20221025/P434', '20221025/P437', '20221025/P600', 
                '20221026/P195', '20221026/P393', '20221026/P430', '20221026/P434', '20221026/P437', '20221026/P600', 
                '20221027/P195', '20221027/P393', '20221027/P430', '20221027/P434', '20221027/P437', '20221027/P600', 
                '20221028/P195', '20221028/P393', '20221028/P430', '20221028/P434', '20221028/P437', '20221028/P600', 
                '20221031/P195', '20221031/P393', '20221031/P430', '20221031/P434', '20221031/P437', '20221031/P600', 
                '20221101/P195', '20221101/P393', '20221101/P430', '20221101/P434', '20221101/P437', '20221101/P600', 
                '20221102/P195', '20221102/P393', '20221102/P430', '20221102/P434', '20221102/P437', '20221102/P600', 
                '20221103/P195', '20221103/P393', '20221103/P430', '20221103/P434', '20221103/P437', '20221103/P600', 
                '20221104/P195', '20221104/P393', '20221104/P430', '20221104/P434', '20221104/P437', '20221104/P600', 
                '20230206/P122', '20230206/P204', '20230206/P428', '20230206/P436', '20230206/P589', '20230206/P791', 
                '20230207/P122', '20230207/P204', '20230207/P428', '20230207/P436', '20230207/P589', '20230207/P791', 
                '20230208/P122', '20230208/P204', '20230208/P428', '20230208/P436', '20230208/P589', '20230208/P791', 
                '20230209/P122', '20230209/P204', '20230209/P428', '20230209/P436', '20230209/P589', '20230209/P791', 
                '20230210/P122', '20230210/P204', '20230210/P428', '20230210/P436', '20230210/P589', '20230210/P791', 
                '20230213/P122', '20230213/P204', '20230213/P428', '20230213/P436', '20230213/P589', '20230213/P791', 
                '20230214/P122', '20230214/P204', '20230214/P428', '20230214/P436', '20230214/P589', '20230214/P791', 
                '20230215/P122', '20230215/P204', '20230215/P428', '20230215/P436', '20230215/P589', '20230215/P791', 
                '20230216/P122', '20230216/P204', '20230216/P428', '20230216/P436', '20230216/P589', '20230216/P791', 
                '20230217/P122', '20230217/P204', '20230217/P428', '20230217/P436', '20230217/P589', '20230217/P791', 
                '20230220/P122', '20230220/P204', '20230220/P428', '20230220/P436', '20230220/P589', '20230220/P791', 
                '20230221/P122', '20230221/P204', '20230221/P428', '20230221/P436', '20230221/P589', '20230221/P791', 
                ]
# see reference_nesting = 1 because cameras were referenced and calibrated only once per session
projectpath = aniposespecific.create_projectstructure(project_name, project_directory, session_list, reference_nesting=1, calibration_nesting=1, output = True)

Anipose Project created or updated: G:\ForagingPlatformsArena_local\Triangulation


In [ ]:
# populate project with videos from source paths

behavior_path = r'G:\ForagingPlatformsArena_local\Behavior'
calibration_path = r'G:\ForagingPlatformsArena_local\Calibration'
reference_path = r'G:\ForagingPlatformsArena_local\Reference'

aniposespecific.populate_project(projectpath, behavior_path, calibration_path, reference_path, separators = r'_|\\|/',copy = False)

In [4]:
# create a config.toml file
config_path = aniposespecific.create_config(projectpath, output = False)

# Add DeepLabCut Models to the Project

In [49]:
# monkeypatch for pretrained DLC models (works only in DLC < v2.3)
dlcspecifics.monkeypatch()

# create dummy video
projectpath = r'G:\ForagingPlatformsArena_local\Triangulation'
os.chdir(projectpath)
url = 'https://gitlab.ruhr-uni-bochum.de/hidalggc/3dposetrackingoffreelymovingpigeons/-/raw/main/models/PigeonSuperModel.png?inline=false'
video_path = utils.create_dummy_videos(projectpath, url)

# create pre-trained model for PigeonSuperModel
beh_config_path, _ = deeplabcut.create_pretrained_project(
    'DLC_PigeonSuperModel_effnet_b0',
    'PigeonSuperModel.com',
    [video_path],
    videotype='mp4',
    model='PigeonSuperModel_effnet_b0',
    analyzevideo=False,
    filtered=False,
    createlabeledvideo=False,
    copy_videos=True,)

# create pre-trained model for Arena Reference
ref_config_path, _ = deeplabcut.create_pretrained_project(
    'DLC_RefModel_HexArena_resnet_50',
    'PigeonSuperModel.com',
    [video_path],
    videotype='mp4',
    model='RefModel_HexArena_resnet_50',
    analyzevideo=False,
    filtered=False,
    createlabeledvideo=False,
    copy_videos=True,)

os.remove(video_path)

Created "G:\ForagingPlatformsArena_local\Triangulation\DLC_RefModel_HexArena_resnet_50-PigeonSuperModel.com-2023-06-04\videos"
Created "G:\ForagingPlatformsArena_local\Triangulation\DLC_RefModel_HexArena_resnet_50-PigeonSuperModel.com-2023-06-04\labeled-data"
Created "G:\ForagingPlatformsArena_local\Triangulation\DLC_RefModel_HexArena_resnet_50-PigeonSuperModel.com-2023-06-04\training-datasets"
Created "G:\ForagingPlatformsArena_local\Triangulation\DLC_RefModel_HexArena_resnet_50-PigeonSuperModel.com-2023-06-04\dlc-models"
Copying the videos
G:\ForagingPlatformsArena_local\Triangulation\DLC_RefModel_HexArena_resnet_50-PigeonSuperModel.com-2023-06-04\videos\logo.mp4
Generated "G:\ForagingPlatformsArena_local\Triangulation\DLC_RefModel_HexArena_resnet_50-PigeonSuperModel.com-2023-06-04\config.yaml"

A new project with name DLC_RefModel_HexArena_resnet_50-PigeonSuperModel.com-2023-06-04 is created at G:\ForagingPlatformsArena_local\Triangulation and a configurable file (config.yaml) is st

177668096B [00:05, 34196312.54B/s]                                 

G:\ForagingPlatformsArena_local\Triangulation\DLC_RefModel_HexArena_resnet_50-PigeonSuperModel.com-2023-06-04\dlc-models\iteration-0\DLC_RefModel_HexArena_resnet_50Jun4-trainset95shuffle1\train\pose_cfg.yaml


In [50]:
# edit DLC models for viterbi filter

edit = {'num_outputs': 10} # this is relevant for viterbi filter
deeplabcut.auxiliaryfunctions.edit_config(beh_config_path, edit);
deeplabcut.auxiliaryfunctions.edit_config(ref_config_path, edit);

# Update Anipose Project Settings

In [53]:
# change config.toml
edits = {
    'motionpype': {
        'ref_model_folder': os.path.dirname(ref_config_path),
        'beh_model_folder': os.path.dirname(beh_config_path), 
        },
    'nesting': 2,
    }

aniposespecific.change_toml(edits, config_path)

# find local and remote anaconda
aniposespecific.find_anaconda(config_path, directories = ['C:\\Users','C:\\ProgramData',], query = 'Anaconda3', overwrite = False, output = False)
aniposespecific.find_anaconda(config_path, directories = [r"\\compute.ikn.psy.rub.de\\C$\\ProgramData",], query = 'Anaconda3', overwrite = False, output = False)

# Process Video Data

## Analyze Behavior Videos

In [38]:
# change model_folder, video_type, videos_raw
videotype = aniposespecific.check_videotype(config_path, 'videos-raw')
config = aniposespecific.read_config(config_path)
settings = {
    'video_extension': videotype,
    'model_folder': config['motionpype']['beh_model_folder'],
    'nesting': 2,
    'pipeline': {
        'videos_raw': 'videos-raw',
        'calibration_videos': 'videos-cal',
        'calibration_results': 'videos-cal',
        'pose_2d': 'pose-2d',
        'pose_2d_filter': 'pose-2d-filtered', # this needs to be different than the directory above
        'pose_3d': 'pose-3d',
        'pose_3d_filter': 'pose-3d-filtered',
        },
    }

In [39]:
# run anipose on independent thread
aniposespecific.execute_anipose(settings, 'analyze', config_path)

Running anipose analyze on Breadnut...


In [2]:
config_path = r"G:\ForagingPlatformsArena_local\Triangulation\config.toml"
aniposespecific.project_overview(config_path)

Project active in G:\ForagingPlatformsArena_local\Triangulation
Found 25 sessions and 25 trials
Found {6} videos per calibration, with 25 out of 25 sessions already calibrated
- average calibration error of 0.78
- bootstraped 95%-CI [0.65 - 0.95]
Found 438 videos, (292 % of expected with 6 cameras and 25 trials)
Found {18, 6} number of behavior videos per trial
Found 144 analyzed videos (32 % of all 438 videos)
Found 80 triangulated trials (320 % of all 25 trials)


## Analyze Reference Videos

In [57]:
# change model_folder, video_type, videos_raw
videotype = aniposespecific.check_videotype(config_path, 'videos-ref')
config = aniposespecific.read_config(config_path)

settings = {
    'model_folder': config['motionpype']['ref_model_folder'],
    'video_extension': videotype,
    'nesting': 1, # change this to access session-wide reference videos
    'pipeline': {
        'videos_raw': 'videos-ref', # changed this to grab ref videos
        'calibration_videos': 'videos-cal',
        'calibration_results': 'videos-cal',
        'pose_2d': 'videos-ref',
        'pose_2d_filter': 'pose-2d-filtered',
        'pose_3d': 'pose-3d',
        'pose_3d_filter': 'pose-3d-filtered',
        },
    }

In [58]:
# run anipose on independent thread
aniposespecific.execute_anipose(settings, 'analyze', config_path)

Running anipose analyze on Breadnut...


### Migrate project to Server for higher CPU

In [16]:
client = r"\\compute.ikn.psy.rub.de"
dest_path = "D$\\UserData\\Guillermo\\ForagingPlatforms_Triangulation_server"
aniposespecific.clone_anipose_project(config_path, client, dest_path, videos = False)

Cloning project: G:\ForagingPlatformsArena_local\Triangulation 
 to remote path: D:\UserData\Guillermo\ForagingPlatforms_Triangulation_server on \\compute.ikn.psy.rub.de


## Viterbi and Median Filters

In [59]:
# change filer settings
settings = {
    'nesting': 1, # change this to 1 to grab ref videos
    'pipeline': {
        'videos_raw': 'videos-raw',
        'calibration_videos': 'videos-cal',
        'calibration_results': 'videos-cal',
        'pose_2d': 'videos-ref', # change this to grab ref videos
        'pose_2d_filter': 'videos-ref-filtered', # change this to grab ref videos
        'pose_3d': 'pose-3d',
        'pose_3d_filter': 'pose-3d-filtered',
        },
    'filter': {
        'enabled': True, 
        'type': 'viterbi', 
        'score_threshold': 0, # disable tracking likelihood 
        'n_back': 10,
        'offset_threshold': 50, 
        'multiprocessing': True
        }, 
    }

In [60]:
# run anipose on remote
command = 'filter'
aniposespecific.execute_anipose(settings, command, config_path, output = False)

Running anipose filter on Breadnut...


In [35]:
# change filer settings
settings = {
    'nesting': 2, # changed this to grab ref videos
    'pipeline': {
        'videos_raw': 'videos-raw',
        'calibration_videos': 'videos-cal',
        'calibration_results': 'videos-cal',
        'pose_2d': 'pose-2d', # changed this to grab ref videos
        'pose_2d_filter': 'pose-2d-filtered',
        'pose_3d': 'pose-3d',
        'pose_3d_filter': 'pose-3d-filtered',
        },
    'filter': {
        'enabled': True, 
        'type': 'viterbi', 
        'score_threshold': 0, # tracking likelihood 
        'n_back': 10,
        'offset_threshold': 50, 
        'multiprocessing': True
        }, 
    }

In [36]:
# run anipose on remote
command = 'filter'
client = r'\\compute.ikn.psy.rub.de'
aniposespecific.execute_anipose(settings, command, config_path, client, output = False)

Running anipose filter on compute-ikn.serverhosting.ruhr-uni-bochum.de...


# Merge Behavior and Reference Data 

In [2]:
# pull progress from remote
config_path = r"G:\ForagingPlatformsArena_local\Triangulation\config.toml"
aniposespecific.pull_cloned_project(config_path)

Pulling project from \\compute.ikn.psy.rub.de\D$\UserData\Guillermo\ForagingPlatforms_Triangulation_server 
 to: G:\ForagingPlatformsArena_local\Triangulation


In [89]:
aniposespecific.project_overview(config_path)

Project active in G:\ForagingPlatformsArena_local\Triangulation
Found 25 sessions and 144 trials
Found {6} videos per calibration, with 25 out of 25 sessions already calibrated
- average calibration error of 0.78
- bootstraped 95%-CI [0.65 - 0.95]
Found 732 videos, (84 % of expected with 6 cameras and 144 trials)
Found {0, 6} number of behavior videos per trial
Found 0 analyzed videos (0 % of all 732 videos)
Found 56 triangulated trials (38 % of all 144 trials)


In [68]:
# change merging settings
settings = {
    'merge': {
        'behavior': 'pose-2d-filtered', 
        'reference': 'ref-filtered', 
        'output':'pose-2d-merged', 
        'nesting': 1, # levels between behavior and reference
        }, 
    'triangulation':{
        'triangulate': True,
        'cam_regex': "_cam([A-Z])$",
        },
    }
aniposespecific.change_toml(settings, config_path)

In [ ]:
aniposespecific.merge_referenceframes(config_path, overwrite = True, output=True)

In [74]:
# 2D Filter merged pose with median
# change filer settings
settings = {
    'nesting': 2, # changed this to grab ref videos
    'pipeline': {
        'videos_raw': 'videos-raw',
        'calibration_videos': 'videos-cal',
        'calibration_results': 'videos-cal',
        'pose_2d': 'pose-2d-merged', # grab merged poses to filter
        'pose_2d_filter': 'pose-2d-merged-filtered',
        'pose_3d': 'pose-3d',
        'pose_3d_filter': 'pose-3d-filtered',
        },
    'filter': {
        'enabled': True, 
        'type': 'medfilt', 
        'score_threshold': 0, # ignore tracking likelihood
        'offset_threshold': 20, # pixel difference to overwrite TODO how big is the scale?
        'medfilt': 15, # at 50Hz this is a 300ms filter window 
        'spline': True, # cubic interpolation
        'multiprocessing': True
        },
    }

In [79]:
aniposespecific.execute_anipose(settings, 'filter', config_path, output = False)

Running anipose filter on Breadnut...


In [65]:
# Data verification
projectpath = os.path.dirname(config_path)
files = utils.scrapdirbystring(projectpath, 'videos-ref-filtered')

Scrapping files in G:\ForagingPlatformsArena_local\Triangulation returned list of size: 150


In [ ]:
for file in files:
    df = pd.read_hdf(file)
    subset = [col for col in df.columns if 'cA' in col or 'cB' in col or 'cC' in col or 'cD' in col or 'cE' in col or 'cF' in col]
    xcols = [col for col in subset if 'x' in col]
    ycols = [col for col in subset if 'y' in col]
    xref = df.loc[:,xcols]
    yref = df.loc[:,ycols]
    plt.scatter(xref, yref);
    plt.scatter(xref.iloc[:,0], yref.iloc[:,0], color = 'red');
    plt.ylim((1440,0))
    plt.title(f'{file}')
    plt.show()

## Calibrate Cameras

In [72]:
# edit calibration parameters and change video_type if necessary
videotype = aniposespecific.check_videotype(projectpath, 'videos-cal')

edits = {
    'video_extension': videotype,
    'pipeline': {
        'videos_raw': 'videos-raw',
        'calibration_videos': 'videos-cal',
        'calibration_results': 'videos-cal',
        'pose_2d': 'pose-2d-merged', # grab merged poses to filter
        'pose_2d_filter': 'pose-2d-merged-filtered',
        'pose_3d': 'pose-3d',
        'pose_3d_filter': 'pose-3d-filtered',
        },
    'calibration': {
        'board_type': 'charuco', 
        'board_size': [7, 7], 
        'board_marker_bits': 4, 
        'board_marker_dict_number': 50, 
        'board_marker_length': 49, 
        'board_square_side_length': 70, 
        'animal_calibration': False, 
        'calibration_init': None,
        'fisheye': False,
        }, 
    'manual_verification': {
        'manually_verify': False,
        },
    }

Found multiple video extensions: toml


In [73]:
# run anipose on independent thread
aniposespecific.execute_anipose(edits, 'calibrate', config_path)

Running anipose calibrate on Breadnut...


In [ ]:
# TODO consider patching calibration errors


# Triangulate Data

Manual trick: upload only the pose-2d-merged-filtered data for the upper half of the project and start a triangulation process. Once started, upload the second half of the pose-2d-merged-filtered data and start a second parallel process. This assumes RAM and CPU ressources would suffice for two parallel instances...

In [83]:
# move project to server
client = r"\\compute.ikn.psy.rub.de"
dest_path = "D$\\UserData\\Guillermo\\ForagingPlatforms_Triangulation_server"
aniposespecific.clone_anipose_project(config_path, client, dest_path, videos = False)

Cloning project: G:\ForagingPlatformsArena_local\Triangulation 
 to remote path: D:\UserData\Guillermo\ForagingPlatforms_Triangulation_server on \\compute.ikn.psy.rub.de


In [ ]:
# delete upper half on compute server

# delete 3d poses on compute

# start triangulation

# clone again

# delete 3d poses on compute

# start second triangulation

In [8]:
# need to specify what data to get (filtered or not, where to find the calibration, and the actual triangulation parameters
settings = {
    'pipeline': {
        'videos_raw': 'videos-raw',
        'calibration_videos': 'videos-cal',
        'calibration_results': 'videos-cal',
        'pose_2d': 'pose-2d-merged', # grab merged poses to filter
        'pose_2d_filter': 'pose-2d-merged-filtered',
        'pose_3d': 'pose-3d',
        'pose_3d_filter': 'pose-3d-filtered',
        },
    'filter': {
        'enabled': True, # enable this to point to 'pose_2d_filter'
        }, 
    'triangulation': {
        'triangulate': True, 
        'cam_regex': '_cam([A-Z])$', 
        'ransac': False, 
        'optim': True, 
        'axes': [['x', 'cB', 'cA'], ['y', 'cA', 'cE']],
        'reference_point': 'cA', 
        'scale_smooth': 4,  # strength of temporal constraint
        'scale_length': 2, # strength of spatial constraint
        'reproj_error_threshold': 50, # in pixels, for robust triangulation
        'score_threshold': 0, # ignore original tracking likelihood
        'n_deriv_smooth': 1, # speed should be smooth
        'constraints': [ # interconnected arena
            ['cA', 'cB'], ['cB', 'cC'], ['cC', 'cD'], ['cD', 'cE'], ['cE', 'cF'], ['cF', 'cA'], 
            ['cA', 'cE'], ['cF', 'cD'], ['cE', 'cC'],['cD', 'cB'],['cC', 'cA'], ['cB', 'cF'],
            ['cA', 'cD'], ['cB', 'cE'], ['cC', 'cF'], 
            ],
        },
    }

In [10]:
# run anipose on remote
command = 'triangulate'
client = r'\\compute.ikn.psy.rub.de'
aniposespecific.execute_anipose(settings, command, config_path, client, output = False)

Running anipose triangulate on compute-ikn.serverhosting.ruhr-uni-bochum.de...


In [7]:
# ETA 
# 60 min GPU per camera at 17 its/s
# 1 min GPU per ref camera
# 15 min calibration per session
# 1 min viterbi filter
# 10 sec viterbi filter
# 70 min merge total
# 10 sec median filter
# 50 min triangulating points at 900 its/s
# 2-8h optimization (+200GB RAM)

In [14]:
# pull progress from remote
config_path = r"G:\ForagingPlatformsArena_local\Triangulation\config.toml"
aniposespecific.pull_cloned_project(config_path)

Pulling project from \\compute.ikn.psy.rub.de\D$\UserData\Guillermo\ForagingPlatforms_Triangulation_server 
 to: G:\ForagingPlatformsArena_local\Triangulation


In [15]:
aniposespecific.project_overview(config_path)

Project active in G:\ForagingPlatformsArena_local\Triangulation
Found 25 sessions and 144 trials
Found {6} videos per calibration, with 25 out of 25 sessions already calibrated
- average calibration error of 0.78
- bootstraped 95%-CI [0.65 - 0.94]
Found 732 videos, (84 % of expected with 6 cameras and 144 trials)
Found {0, 6} number of behavior videos per trial
Found 0 analyzed videos (0 % of all 732 videos)
Found 122 triangulated trials (84 % of all 144 trials)
